In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
if ('../py' not in sys.path): sys.path.insert(1, '../py')

In [3]:

from pandas import DataFrame

from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

from is_header_sgd_classifier import IsHeaderSgdClassifier
ihu = IsHeaderSgdClassifier(ha=ha, cu=cu, verbose=False)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)

    from section_utils import SectionUtilities
    su = SectionUtilities(verbose=False)

    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    slrcu.build_pos_logistic_regression_elements()

    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(e.__class__)

In [5]:

%run ../load_magic/dataframes.py
from datetime import datetime
from urllib.parse import urlparse, parse_qs
from urllib.error import HTTPError, URLError
from IPython.display import clear_output
import random
import requests
print(f'Last run on {datetime.now()}')

Last run on 2022-07-18 07:35:17.708511



----

In [38]:

query_dict = {
    'ff': 'Location,LessThanOrEq,42.4173,-71.684,100',
    'zip': 'Clinton, MA 01510',
    'rd': '100',
    'miles': true,
    'remote': true,
    'srch': 'data'
}
prefix_str = 'https://jobs.insightglobal.com/find_a_job/massachusetts/clinton/?'
url_str = prefix_str + '&'.join([f'{k}=' + v.replace(' ', '+') for k, v in query_dict.items()])
response_obj = requests.get(url_str)
html_str = response_obj.text
file_path = '../data/html/insight_searchresults.html'
with open(file_path, 'w', encoding='utf-8') as f:
    print(html_str, file=f)
page_soup = wsu.get_page_soup(file_path)
link_css = 'div.result > div.job-title > a'
link_soups_list = page_soup.select(link_css)
len(link_soups_list)

10

In [42]:

url_strs_list = []
rows_list = []
for link_soup in link_soups_list:
    url_str = link_soup['href']
    row_dict = {k: v[0] for k, v in parse_qs(urlparse(url_str).query).items()}
    row_dict['url_str'] = url_str
    rows_list.append(row_dict)
df = pd.DataFrame(rows_list)
url_strs_list = df.url_str.tolist()
len(url_strs_list)

10

In [45]:

'https://jobs.insightglobal.com' + url_strs_list[0]

'https://jobs.insightglobal.com/find_a_job/texas/plano/sr-data-analyst-data-scientist/job-153005/'

In [46]:

import webbrowser

webbrowser.open('https://jobs.insightglobal.com' + url_strs_list[0], new=2)

In [47]:

nojobs_css = '#results-main-data > div.results-panel > div.no-jobs'
nojobs_regex = re.compile('Your search has returned 0 results')

In [30]:

files_list = []
for url_str in url_strs_list:
    url_str = 'https://jobs.insightglobal.com' + url_str
    response_obj = requests.get(url_str, cookies=cookies_dict, headers=headers_dict)
    webpage_html = response_obj.text
    match_obj = nojobs_regex.search(webpage_html)
    if match_obj:
        cu.set_is_closed(file_name, verbose=False)
    else:
    page_title = eval(f'{{{match_obj.group()}}}')['jobTitleText']
    file_name = re.sub(r'[^A-Za-z0-9]+', ' ', page_title).strip().replace(' ', '_')
    jobListingId = parse_qs(urlparse(url_str).query).get('jobListingId', [''])[0]
    if len(jobListingId):
        file_name = f'{jobListingId}_{file_name}.html'
    else:
        file_name = f'{file_name}.html'
        file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
        if os.path.isfile(file_path):
            file_name = datetime.now().strftime('%Y%m%d%H%M%S%f') + f'_{file_name}'
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    if not os.path.isfile(file_path):
        with open(file_path, 'w', encoding=s.encoding_type) as f:
            print(f'Saving to {file_path}')
            f.write('<html><head><title>')
            f.write(page_title)
            f.write('</title></head><body><div id="jobDescriptionText">')
            match_obj = desc_regex.search(webpage_html)
            html_str = eval(f'{{{match_obj.group()}}}')['job']['description']
            f.write(html_str)
            f.write('</div></body></html>')
        files_list.append(file_name)
    cu.ensure_filename(file_name, verbose=False)
    cu.set_posting_url(file_name, url_str, verbose=False)
print(f'Fileing {len(files_list)} postings complete')

Fileing 0 postings complete


In [31]:

cu.ensure_navigableparent('END', verbose=False)
for file_name in files_list:
    file_path = os.path.join(cu.SAVES_HTML_FOLDER, file_name)
    page_soup = wsu.get_page_soup(file_path)
    row_div_list = page_soup.find_all(name='body')
    for div_soup in row_div_list:
        child_strs_list = ha.get_navigable_children(div_soup, [])
        assert child_strs_list, f'Something is wrong with {file_name}'
        cu.populate_from_child_strings(child_strs_list, file_name, verbose=False)
files_list

[]